In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import _pickle as pickle

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [6]:
def getCIFAR10_test(path = "cifar-10-batches-py/test_batch"):
    with open(path, 'rb') as f:
        raw_data = pickle.load(f, encoding = 'bytes')
        X, y = np.array(raw_data[b'data'], dtype = np.float32)/255, np.array(raw_data[b'labels'])
        X = np.transpose(np.reshape(X, [-1,3,32,32]), (0,2,3,1))
        print(X.shape)
        return(X,y)
        
def getCIFAR10_train(path = "cifar-10-batches-py/"):
    train_data_set = np.zeros([50000, 3,32, 32], dtype = np.float32)
    train_labels = np.zeros([50000,], dtype= int)
    
    file_name = path + "data_batch_"
    for i in range(1,6):
        file = open(file_name + str(i), 'rb')
        raw_data = pickle.load(file, encoding='bytes')
        
        batch_labels = np.array(raw_data[b'labels'])
        batch_imgs= np.array(raw_data[b'data'], dtype = np.float32) / 255 #normalize
        batch_imgs = np.reshape(batch_imgs, [-1,3,32,32])

        #append
        m = len(batch_imgs)
        train_data_set[m*(i-1):m*i, :] = batch_imgs
        train_labels[m*(i-1):m*i] = batch_labels
        
        file.close()
    del batch_imgs,batch_labels
    train_data_set=train_data_set.transpose(0,2,3,1)
    labels_dict = {0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9:"truck"}
    return (train_data_set, train_labels, labels_dict)

In [4]:
def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features['x'], [-1,32,32,3])

    #Convolutional layer 1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32,
        kernel_size = [5,5],
        padding = "same",
        activation = None)
    batnorm1  = tf.layers.batch_normalization(inputs = conv1)
    act1 = tf.nn.elu(batnorm1)
    #Add a max pooling layer 1
    pool1 = tf.layers.max_pooling2d(
        inputs = act1, pool_size = [2,2], strides = 2)

    #Convolutional layer 2 and max pooling layer 2
    conv2_branch1 = tf.layers.conv2d(
        inputs = pool1,
        filters = 48,
        kernel_size = [1,1],
        padding = "same",
        activation = None)
    batnorm2_branch1 = tf.layers.batch_normalization(inputs = conv2_branch1)
    act2_branch1 =  tf.nn.elu(batnorm2_branch1)
    
    conv2_branch2 = tf.layers.conv2d(
        inputs = act2_branch1,
        filters = 64,
        kernel_size = [5,5],
        padding = "same",
        activation = None)
    batnorm2_branch2 = tf.layers.batch_normalization(inputs = conv2_branch2)
    act2_branch2 = tf.nn.elu(batnorm2_branch2)
    pool2 = tf.layers.max_pooling2d(
        inputs = act2_branch2, pool_size = [2,2], strides = 2)

    #Convolutional layer 3 and maxpooling layer 3
    conv3_branch1 = tf.layers.conv2d(
        inputs = pool2,
        filters = 96,
        kernel_size = [1,1],
        padding = "same",
        activation = None) 
    batnorm3_branch1 =  tf.layers.batch_normalization(inputs = conv3_branch1)
    act3_branch1 =  tf.nn.elu(batnorm3_branch1)
    
    conv3_branch2 = tf.layers.conv2d(
        inputs = act3_branch1,
        filters = 128,
        kernel_size = [5,5],
        padding = "same",
        activation = None)
    batnorm3_branch2 = tf.layers.batch_normalization(inputs = conv3_branch2)
    act3_branch2 =  tf.nn.elu(batnorm3_branch2)
    pool3 = tf.layers.max_pooling2d(
        inputs = act3_branch2, pool_size = [2,2], strides = 2)
    
    #Dense Layer (start using fully connected)
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    pool3_flatten = tf.reshape(pool3, [-1,4*4*128])
    dense = tf.layers.dense(inputs = pool3_flatten, units = 2048, activation = tf.nn.elu)
    dropout = tf.layers.dropout(
        inputs= dense, rate = .4, training= is_training)
    
    #Logits layer fully connected forward then softmax
    logits = tf.layers.dense(inputs = dropout, units = 10)

    predictions = {
        "classes":tf.argmax(input = logits, axis = 1),
        "probabilities": tf.nn.softmax(logits, name = "softmax_tensor") 
    }
    #check if making prediction. if yes then no backwardprop
    if(mode == tf.estimator.ModeKeys.PREDICT):
        return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)

    #Calculate Loss func
    onehot_labels = tf.one_hot(indices = tf.cast(labels, tf.int32), depth = 10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits = logits)

    #train
    if(is_training):
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1 = .9, beta2 = .995, epsilon=1e-07)
        train_op = optimizer.minimize(
            loss = loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
        labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
def main(unused_argv):
    eval_data, eval_labels = getCIFAR10_test()
    train_data, train_labels, label_name = getCIFAR10_train()
    
    #create the estimator
    cifar_classifier = tf.estimator.Estimator(
        model_fn = cnn_model_fn, model_dir = "/home/thanh1998tom/assignment1/Tensorflow/tmp")
    
    #Set up logging for predictions
    #Log the values in the "Softmax" tensor with label probabilities
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    
    #train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x= {"x": train_data},
        y = train_labels,
        batch_size = 100,
        num_epochs = None,
        shuffle = True
    )
    
    cifar_classifier.train(
        input_fn = train_input_fn,
        steps = 10000,
        hooks = [logging_hook]
    )
    
    #evaluate the test set
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = cifar_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)
    
    pred_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {"x":test_data},
        num_epochs = 1,
        shuffle = False
    )
    pred_results = list(cifar_classifier.predict(input_fn = pred_input_fn))
    pred_classes = np.reshape(np.array([p["classes"] for p in  pred_results]), (10000,1))
    counts = np.reshape(np.arange(0,10000)+1, (10000,1))
    sol = np.append(counts, pred_classes, axis = 1)
    np.savetxt("mnist_test_pred.csv", sol,fmt = '%i', delimiter=',')
    

In [ ]:
tf.app.run()

(10000, 32, 32, 3)
INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_save_summary_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_model_dir': '/home/thanh1998tom/assignment1/Tensorflow/tmp', '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000}


INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_save_summary_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_model_dir': '/home/thanh1998tom/assignment1/Tensorflow/tmp', '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:probabilities = [[ 0.00800744  0.00606695  0.05413911  0.18651192  0.24143754  0.15932362
   0.03487735  0.30079061  0.00220032  0.00664508]
 [ 0.02240455  0.0103532   0.14759588  0.2718637   0.09602508  0.21602333
   0.03593089  0.15906271  0.01485622  0.0258845 ]
 [ 0.03584781  0.01319314  0.49631965  0.10847425  0.17964748  0.04225136
   0.04341796  0.04386032  0.01196166  0.02502639]
 [ 0.01366858  0.00609696  0.10617041  0.11887101  0.3260428   0.08332742
   0.15173513  0.18340026  0.00382817  0.00685927]
 [ 0.02785593  0.0065314   0.13321517  0.11587255  0.1693414   0.07694577
   0.04253628  0.37146366  0.00823145  0.04800637]
 [ 0.23327598  0.04531314  0.1685764   0.11693007  0.11391731  0.04670718
   0.02323774  0.11852032  0.04213953  0.09138227]
 [ 0.00203529  0.11036853  0.01955971  0.31573647  0.03209799  0.11347209
   0.10564963  0.13468738  0.00380117  0.16259173]
 [ 0.08542404  0.50877321  0.05176356  0.07600441  0.04266421  0.0335531
   0.01587581  0.086

INFO:tensorflow:step = 101, loss = 1.61053 (56.578 sec)


INFO:tensorflow:step = 101, loss = 1.61053 (56.578 sec)


INFO:tensorflow:probabilities = [[ 0.06832527  0.11575262  0.01898099  0.00920077  0.0145133   0.00131078
   0.05794193  0.00196599  0.55119854  0.16080976]
 [ 0.00628337  0.00264185  0.08008391  0.06762099  0.50379771  0.04435091
   0.26891083  0.01383245  0.01127068  0.00120734]
 [ 0.0003184   0.00247748  0.10869848  0.1912111   0.06820816  0.27743727
   0.00541541  0.3457312   0.00018815  0.00031445]
 [ 0.03935254  0.00051653  0.15071458  0.12535973  0.07275459  0.0919539
   0.00296582  0.4768815   0.01828563  0.02121528]
 [ 0.00413403  0.00776939  0.00519109  0.01581484  0.01101233  0.01129832
   0.00206223  0.92013794  0.00400007  0.0185797 ]
 [ 0.0457697   0.00738406  0.00085633  0.00038005  0.00032678  0.00002626
   0.00352949  0.00002973  0.93108535  0.01061224]
 [ 0.02084733  0.04154218  0.09495421  0.06599379  0.01003077  0.03825997
   0.03608155  0.31584743  0.0328974   0.34354523]
 [ 0.29657227  0.00224933  0.00556355  0.00013724  0.00244412  0.0000139
   0.00013833  0.0000

INFO:tensorflow:probabilities = [[ 0.06832527  0.11575262  0.01898099  0.00920077  0.0145133   0.00131078
   0.05794193  0.00196599  0.55119854  0.16080976]
 [ 0.00628337  0.00264185  0.08008391  0.06762099  0.50379771  0.04435091
   0.26891083  0.01383245  0.01127068  0.00120734]
 [ 0.0003184   0.00247748  0.10869848  0.1912111   0.06820816  0.27743727
   0.00541541  0.3457312   0.00018815  0.00031445]
 [ 0.03935254  0.00051653  0.15071458  0.12535973  0.07275459  0.0919539
   0.00296582  0.4768815   0.01828563  0.02121528]
 [ 0.00413403  0.00776939  0.00519109  0.01581484  0.01101233  0.01129832
   0.00206223  0.92013794  0.00400007  0.0185797 ]
 [ 0.0457697   0.00738406  0.00085633  0.00038005  0.00032678  0.00002626
   0.00352949  0.00002973  0.93108535  0.01061224]
 [ 0.02084733  0.04154218  0.09495421  0.06599379  0.01003077  0.03825997
   0.03608155  0.31584743  0.0328974   0.34354523]
 [ 0.29657227  0.00224933  0.00556355  0.00013724  0.00244412  0.0000139
   0.00013833  0.0000

In [ ]:
import pandas as pd
def pred(unused_argv):
    test_data = np.array(pd.read_csv('test.csv', sep = ','), dtype = np.float32)
    mnist_classifier = tf.estimator.Estimator(
        model_fn = cnn_model_fn, model_dir = "/tmp/mnist_convnet_model/")
    
    pred_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {"x":test_data},
        num_epochs = 1,
        shuffle = False
    )
    pred_results = list(mnist_classifier.predict(input_fn = pred_input_fn))
    pred_classes = np.reshape(np.array([p["classes"] for p in  pred_results]), (28000,1))
    counts = np.reshape(np.arange(0,28000)+1, (28000,1))
    sol = np.append(counts, pred_classes, axis = 1)
    np.savetxt("mnist_test_pred.csv", sol,fmt = '%i', delimiter=',')
    print(len(pred_classes))
    
    return pred_classes

In [ ]:
pred_results = tf.app.run(pred)